In [58]:
import pandas as pd
import csv
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda, LSTM, Dense, Dropout, Input
import os

## Data Preprocessing

In [2]:
rows = []
csv_path = 'data/id_genres.csv'
with open(csv_path, 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    
    header = next(csv_reader)[0].split('\t')
    print(header)
    for row in csv_reader:
        temp = row[0].split('\t')
        if len(row) > 2:
            temp.extend(row[2:])
        rows.append(temp)

['id', 'genres']


In [3]:
header = ['id', 'genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7']

In [5]:
df = pd.DataFrame(rows, columns=header)
df

,id,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,0009fFIM1eYThaPg,pop,None,None,None,None,None,None
1,0010xmHR6UICBOYT,underground hip hop,None,None,None,None,None,None
2,002Jyd0vN4HyCpqL,hard rock,classic rock,None,None,None,None,None
3,006TYKNjNxWjfKjy,symphonic metal,symphonic power metal,None,None,None,None,None
4,007LIJOPQ4Sb98qV,post-punk,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
109264,zzyyPUs7hC9Nz2e1,hardcore punk,hardcore,None,None,None,None,None
109265,zzz0n04uuTUA7fNh,pop,None,None,None,None,None,None
109266,zzzj3LYaZtYtbzSr,singer-songwriter,None,None,None,None,None,None
109267,zzznMjZAKnJJXQSj,synthpop,pop,None,None,None,None,None


In [46]:
melt = pd.melt(df, id_vars=['id'], value_vars=df.columns[1:])
allocc = melt['value'].value_counts()
top10occ = melt['value'].value_counts().head(10)
top10occ

value
pop                 16810
rock                15947
electronic           8855
indie rock           5730
alternative rock     5292
folk                 4559
soul                 3985
classic rock         3809
metal                3639
rap                  3623
Name: count, dtype: int64

In [39]:
top10genres = set(top10occ.keys())
sum(top10occ.values)

72249

In [40]:
users_with_top10 = melt[melt['value'].isin(top10genres)]['id'].drop_duplicates()

In [41]:
filtered_users = df[df['id'].isin(users_with_top10)]

In [42]:
filtered_users

,id,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,0009fFIM1eYThaPg,pop,None,None,None,None,None,None
2,002Jyd0vN4HyCpqL,hard rock,classic rock,None,None,None,None,None
5,00CH4HJdxQQQbJfu,indie rock,shoegaze,experimental,None,None,None,None
6,00DZ3XCAQb2FdCc6,ambient,electronic,None,None,None,None,None
8,00IeldeA9ijZOL0P,pop,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
109262,zzx8CWdM7qkxKQpC,indie rock,None,None,None,None,None,None
109263,zzyb5LvKJTWLVnrk,soul,mpb,funk,None,None,None,None
109265,zzz0n04uuTUA7fNh,pop,None,None,None,None,None,None
109267,zzznMjZAKnJJXQSj,synthpop,pop,None,None,None,None,None


In [55]:
final_df = pd.DataFrame()
final_df['id'] = filtered_users['id']

In [56]:
final_df['genre'] = df.iloc[:, 1:].apply(lambda row: next((x for x in row if x in top10genres), None), axis=1)

In [57]:
final_df

,id,genre
0,0009fFIM1eYThaPg,pop
2,002Jyd0vN4HyCpqL,classic rock
5,00CH4HJdxQQQbJfu,indie rock
6,00DZ3XCAQb2FdCc6,electronic
8,00IeldeA9ijZOL0P,pop
...,...,...
109262,zzx8CWdM7qkxKQpC,indie rock
109263,zzyb5LvKJTWLVnrk,soul
109265,zzz0n04uuTUA7fNh,pop
109267,zzznMjZAKnJJXQSj,pop


In [68]:
directory_path = 'data/lyrics-20231007T122358Z-001/lyrics'
file_prefixes = final_df['id'].tolist()
for prefix in file_prefixes:
    file_pattern = prefix + '.txt'
    file_path = os.path.join(directory_path, file_pattern)
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            content = file.read()
            lyrics.append(content)

FileNotFoundError: [Errno 2] No such file or directory: 'data/lyrics-20231007T122358Z-001/lyrics\\BEGbqj0SS21gd9L3.txt'

In [69]:
f = os.listdir(directory_path)
len(f)

22110